In [1]:
from hail import Table
import os
import pprint
from pprint import pformat
import argparse
import hail as hl
import pandas as pd
import numpy as np
import pyspark
from pprint import pformat
import json
import sys
import re
from pathlib import Path
import logging
from typing import Any, Counter, List, Optional, Tuple, Union, Dict,Set
import uuid
import json
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from gnomad.utils.plotting import *
from typing import Set, Tuple
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
import warnings; 
warnings.simplefilter('ignore')
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
#plot_dir = "/home/ubuntu/data/tmp"
plot_dir="/lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"
lustre_dir = "file:///lustre/scratch123/teams/hgi/mercury/megaWES-variantqc"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=lustre_dir, local_tmpdir=lustre_dir, default_reference="GRCh38")
output_notebook()

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.7
SparkUI available at http://spark-master:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/ubuntu/data/scripts/exomeQC-hail-gnomad/notebooks/hail-20210701-2255-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

In [4]:
mt=hl.read_matrix_table(f'{lustre_dir}/variant_qc/MegaWESSanger_cohorts_sampleQC_filtered_split.mt')
mt=hl.read_matrix_table(f'{lustre_dir}/MegaWESSanger_cohorts_sampleQC_filtered.mt')

In [5]:
mt.count()

(28803842, 93307)

In [6]:
ht=mt.rows()

In [7]:
ht.count()

31316024

In [8]:
mt = mt.select_entries(
        GT=hl.unphased_diploid_gt_index_call(mt.GT.n_alt_alleles()))

In [9]:
mt.count()

(31316024, 93307)

In [10]:
ht = ht.transmute(**ht.info)


In [11]:
ht.count()

31316024

In [2]:
ht=hl.read_table(f'{lustre_dir}/variant_qc/MegaWES_for_RF_all_cols.ht')

In [3]:
ht.count()

28803842

In [5]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'MQ': float64 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': float64 
    'FS': float64 
    'DP': int32 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'ac_raw': int64 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [8]:
ht.omni.show()

,,
locus,alleles,omni
locus<GRCh38>,array<str>,bool
chr1:12938,"[""GCAAA"",""G""]",NA
chr1:13024,"[""G"",""A""]",NA
chr1:13087,"[""A"",""G""]",NA
chr1:13116,"[""T"",""C""]",NA
chr1:13130,"[""C"",""T""]",NA
chr1:13151,"[""G"",""C""]",NA
chr1:13164,"[""G"",""A""]",NA
chr1:13176,"[""G"",""T""]",NA


In [7]:
ht.transmitted_singleton.show()

,,
locus,alleles,transmitted_singleton
locus<GRCh38>,array<str>,bool
chr1:12938,"[""GCAAA"",""G""]",false
chr1:13024,"[""G"",""A""]",false
chr1:13087,"[""A"",""G""]",false
chr1:13116,"[""T"",""C""]",false
chr1:13130,"[""C"",""T""]",false
chr1:13151,"[""G"",""C""]",false
chr1:13164,"[""G"",""A""]",false
chr1:13176,"[""G"",""T""]",false
